In [1]:
import pandas as pd
df = pd.read_csv("dataset\ILPD\ilpd.csv")
df.head()

,age,gender,tb,db,alkphos,sgpt,sgot,tp,alb,ag_ratio,class
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [2]:
LD, NLD = df['class'].value_counts()
print('Number of patients diagnosed with liver disease: ',LD)
print('Number of patients not diagnosed with liver disease: ',NLD)

Number of patients diagnosed with liver disease:  416
Number of patients not diagnosed with liver disease:  167


In [3]:
df['class'].value_counts()

1    416
2    167
Name: class, dtype: int64

In [5]:
df['class'] = df['class'].apply(lambda x: 0 if x == 2 else x)

In [6]:
df['class'].value_counts()

1    416
0    167
Name: class, dtype: int64

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

# Load your dataset
data = df.copy().dropna()

# Identify non-numeric columns and encode them using Label Encoding
non_numeric_columns = data.select_dtypes(include=['object']).columns
label_encoders = {}

for col in non_numeric_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Assuming 'class' is the column with binary labels (0 for Healthy, 1 for Disease)
X = data.drop('class', axis=1)
y = data['class']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a logistic regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Predict the probabilities of disease
risk_probabilities = model.predict_proba(X_test)[:, 1]

# Calculate risk factors (percentages) based on predicted probabilities
risk_factors = risk_probabilities * 100

# Print the risk factors
for i, risk_factor in enumerate(risk_factors):
    print(f"Sample {i + 1}: Risk Factor = {risk_factor:.2f}%")

# Optionally, calculate and print the ROC AUC score
roc_auc = roc_auc_score(y_test, risk_probabilities)
print(f"ROC AUC Score: {roc_auc:.4f}")


Sample 1: Risk Factor = 71.20%
Sample 2: Risk Factor = 88.73%
Sample 3: Risk Factor = 58.22%
Sample 4: Risk Factor = 67.49%
Sample 5: Risk Factor = 68.31%
Sample 6: Risk Factor = 69.50%
Sample 7: Risk Factor = 56.21%
Sample 8: Risk Factor = 54.26%
Sample 9: Risk Factor = 87.21%
Sample 10: Risk Factor = 93.05%
Sample 11: Risk Factor = 86.19%
Sample 12: Risk Factor = 51.91%
Sample 13: Risk Factor = 87.50%
Sample 14: Risk Factor = 62.49%
Sample 15: Risk Factor = 89.47%
Sample 16: Risk Factor = 62.69%
Sample 17: Risk Factor = 84.58%
Sample 18: Risk Factor = 77.16%
Sample 19: Risk Factor = 76.98%
Sample 20: Risk Factor = 63.50%
Sample 21: Risk Factor = 74.99%
Sample 22: Risk Factor = 88.99%
Sample 23: Risk Factor = 68.86%
Sample 24: Risk Factor = 64.94%
Sample 25: Risk Factor = 67.89%
Sample 26: Risk Factor = 65.96%
Sample 27: Risk Factor = 58.86%
Sample 28: Risk Factor = 48.15%
Sample 29: Risk Factor = 54.33%
Sample 30: Risk Factor = 81.02%
Sample 31: Risk Factor = 95.96%
Sample 32: Risk F

c:\Users\Admin\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

# Load your dataset
data = df.copy().dropna()

# Identify non-numeric columns and encode them using Label Encoding
non_numeric_columns = data.select_dtypes(include=['object']).columns
label_encoders = {}

for col in non_numeric_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Assuming 'class' is the column with binary labels (0 for Healthy, 1 for Disease)
X = data.drop('class', axis=1)
y = data['class']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a logistic regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Predict the probabilities of disease
risk_probabilities = model.predict_proba(X_test)[:, 1]

# Calculate risk factors (percentages) based on predicted probabilities
risk_factors = risk_probabilities * 100

# Print the risk factors with original class labels
for i, (risk_factor, original_label) in enumerate(zip(risk_factors, y_test)):
    if original_label == 0:
        label = 'Healthy'
    else:
        label = 'Disease'
    print(f"Sample {i + 1}: Risk Factor = {risk_factor:.2f}%, Original Label = {label}")

# Optionally, calculate and print the ROC AUC score
roc_auc = roc_auc_score(y_test, risk_probabilities)
print(f"ROC AUC Score: {roc_auc:.4f}")


Sample 1: Risk Factor = 71.20%, Original Label = Healthy
Sample 2: Risk Factor = 88.73%, Original Label = Disease
Sample 3: Risk Factor = 58.22%, Original Label = Healthy
Sample 4: Risk Factor = 67.49%, Original Label = Disease
Sample 5: Risk Factor = 68.31%, Original Label = Healthy
Sample 6: Risk Factor = 69.50%, Original Label = Disease
Sample 7: Risk Factor = 56.21%, Original Label = Healthy
Sample 8: Risk Factor = 54.26%, Original Label = Healthy
Sample 9: Risk Factor = 87.21%, Original Label = Disease
Sample 10: Risk Factor = 93.05%, Original Label = Disease
Sample 11: Risk Factor = 86.19%, Original Label = Disease
Sample 12: Risk Factor = 51.91%, Original Label = Healthy
Sample 13: Risk Factor = 87.50%, Original Label = Disease
Sample 14: Risk Factor = 62.49%, Original Label = Healthy
Sample 15: Risk Factor = 89.47%, Original Label = Healthy
Sample 16: Risk Factor = 62.69%, Original Label = Healthy
Sample 17: Risk Factor = 84.58%, Original Label = Disease
Sample 18: Risk Factor 

c:\Users\Admin\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
